<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0
    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-21 09:29:58
-------------------
qualified stocks: 96
with latest results: 33
still star stocks: 19
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  55.89 K
Current:  1.27 C
-------------------
Today PnL: 11.81 K (0.09%)
Current PnL: -16.77 L (-11.89%)
CY Booked + Current PnL: -13.61 L (-9.65%)
-------------------
Total profit:  4.32 L
Total loss:  -21.09 L
-------------------
Total Booked + Current PnL: 13.37 L (11.72%)
Total Booked PnL: 30.14 L (26.43%)
Curr Year Booked PnL: 3.16 L (2.49%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.07 C
Est FTT PnL: 79.90 L (62.87%)
-------------------
Est LTT:  2.68 C
Est LTT PnL: 1.41 C (111.24%)
Deployed:  1.14 C
Current:  1.27 C
CAGR/XIRR %: 7.73%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SBILIFE,1496.49,1924.99,-0.48,H-LC,73.96,147907.0,22202.0,13785.0,0.17,17.66,9.32,28.63,159.0,1.61,1.17,31.77,AR,ATH,INSURANCE
6,ANGELONE,2328.67,3033.00,11.80,H-SC,90.62,182208.0,30844.0,14941.0,0.94,20.38,8.20,30.25,149.0,2.06,1.44,44.21,X40N,NTT,FINANCE
63,RELIANCE,1288.53,1526.00,4.23,X-LC,80.21,224771.0,21183.0,16341.0,-0.14,10.40,7.27,18.43,161.0,1.30,1.78,22.44,XY25,NTT,REFINERIES
57,PIDILITIND,2504.06,3576.00,-18.44,X-LC,33.33,90324.0,15202.0,16954.0,0.27,20.24,18.77,42.81,121.0,0.90,0.71,13.28,X40,BTT,CHEMICALS
71,SBIN,760.30,863.00,-14.39,M-LC,22.92,208542.0,7062.0,20145.0,0.17,3.51,9.66,13.51,60.0,0.35,1.65,16.54,XY25,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MASFIN,326.60,399.50,-24.12,H-SC,52.08,86220.0,-11760.0,33626.0,5.80,-12.00,39.00,22.32,102.0,-0.35,0.68,25.05,XR,ATH,FINANCE
21,COFFEEDAY,59.14,80.00,-57.75,L-SC,97.92,63226.0,-50323.0,90375.0,2.68,-44.32,142.94,35.27,181.0,-0.56,0.50,54.75,XR,NTT,HOTELS
48,LALPATHLAB,2798.50,3545.00,-31.49,H-SC,59.38,131928.0,3197.0,31148.0,1.66,2.48,23.61,26.68,38.0,0.10,1.04,24.13,X40N,NTT,HEALTHCARE
94,WHIRLPOOL,1151.42,2270.00,-33.65,M-SC,98.96,166992.0,20762.0,121303.0,1.50,14.20,72.64,97.15,28.0,0.17,1.32,42.66,X40,NTT,DURABLES
20,COALINDIA,406.65,512.15,8.40,L-LC,44.79,150997.0,536.0,38504.0,1.34,0.36,25.50,25.94,85.0,0.01,1.19,15.22,XY25,ATH,MINING


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SATIN,199.77,274.0,-18.53,H-SC,47.92,162051.0,-33923.0,106743.0,-1.85,-17.31,65.87,37.16,106.0,-0.32,1.28,21.02,XY24,NTT,FINANCE
85,TRIDENT,37.35,48.0,49.12,M-SC,62.50,71581.0,-20674.0,46979.0,-1.66,-22.41,65.63,28.51,207.0,-0.44,0.57,20.65,XR,NTT,TEXTILES
24,DEN,52.79,75.0,96.89,L-SC,38.54,55504.0,-28960.0,64496.0,-1.25,-34.29,116.20,42.07,157.0,-0.45,0.44,17.87,AR,NTT,ENTERTAINMENT
41,INDUSINDBK,1354.50,1800.0,-40.59,H-MC,7.29,44075.0,-33132.0,58523.0,-1.15,-42.91,132.78,32.89,44.0,-0.57,0.35,21.38,XR,NTT,BANKS
30,GREENPANEL,375.16,537.0,172.61,M-SC,55.21,121629.0,-61449.0,140421.0,-1.12,-33.56,115.45,43.14,109.0,-0.44,0.96,14.41,XY24,NTT,MISC


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIAMART,2311.97,4911.36,-51.47,H-SC,60.42,104786.0,3059.0,111314.0,0.76,3.01,106.23,112.43,1.0,0.03,0.83,23.67,AR,ATH,MISC
83,TCS,3928.29,4998.00,-15.84,X-LC,61.46,189562.0,-22566.0,80336.0,0.35,-10.64,42.38,27.23,3.0,-0.28,1.50,8.63,X40,BTT,IT
82,TATAMOTORS,776.02,1065.00,-52.03,M-LC,91.67,191635.0,-14010.0,90586.0,0.59,-6.81,47.27,37.24,4.0,-0.15,1.51,24.73,XY24,NTT,AUTO
10,AWL,342.88,485.00,-69.16,H-MC,4.17,167354.0,-50375.0,140628.0,0.46,-23.14,84.03,41.45,4.0,-0.36,1.32,11.79,XY24,NTT,FMCG
42,INFY,1432.81,2275.00,-13.14,X-LC,68.75,216743.0,17582.0,99485.0,-0.08,8.83,45.90,58.78,6.0,0.18,1.71,13.16,X40,BTT,IT


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ANGELONE,2328.67,3033.00,11.80,H-SC,90.62,182208.0,30844.0,14941.0,0.94,20.38,8.20,30.25,149.0,2.06,1.44,44.21,X40N,NTT,FINANCE
57,PIDILITIND,2504.06,3576.00,-18.44,X-LC,33.33,90324.0,15202.0,16954.0,0.27,20.24,18.77,42.81,121.0,0.90,0.71,13.28,X40,BTT,CHEMICALS
58,POLYCAB,5005.92,7541.35,7.93,H-MC,85.42,168882.0,28716.0,42271.0,0.30,20.49,25.03,50.65,56.0,0.68,1.33,28.89,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,-2.58,M-LC,83.33,188389.0,37183.0,28484.0,-0.69,24.59,15.12,43.43,48.0,1.31,1.49,30.58,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,LUXIND,1448.19,2442.00,42.63,M-SC,65.62,147725.0,10147.0,84262.0,-0.67,7.38,57.04,68.62,66.0,0.12,1.17,28.07,X40N,NTT,TEXTILES
6,ANGELONE,2328.67,3033.00,11.80,H-SC,90.62,182208.0,30844.0,14941.0,0.94,20.38,8.20,30.25,149.0,2.06,1.44,44.21,X40N,NTT,FINANCE
20,COALINDIA,406.65,512.15,8.40,L-LC,44.79,150997.0,536.0,38504.0,1.34,0.36,25.50,25.94,85.0,0.01,1.19,15.22,XY25,ATH,MINING
58,POLYCAB,5005.92,7541.35,7.93,H-MC,85.42,168882.0,28716.0,42271.0,0.30,20.49,25.03,50.65,56.0,0.68,1.33,28.89,X40N,ATH,CABLES
29,GILLETTE,8109.44,10330.69,0.44,H-SC,82.29,148206.0,10346.0,27418.0,0.17,7.50,18.50,27.39,64.0,0.38,1.17,29.74,X40,ATH,FMCG


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,-3.33,H-SC,17.71,108556.0,6552.0,38472.0,0.26,6.42,35.44,44.14,31.0,0.17,0.86,23.71,XR,ATH,MISC
12,BANKINDIA,113.49,190.00,-33.09,M-MC,16.67,132065.0,2913.0,84152.0,-0.10,2.26,63.72,67.42,57.0,0.03,1.04,28.44,XR,NTT,BANKS
39,INDIAMART,2311.97,4911.36,-51.47,H-SC,60.42,104786.0,3059.0,111314.0,0.76,3.01,106.23,112.43,1.0,0.03,0.83,23.67,AR,ATH,MISC
95,WIPRO,243.46,420.00,-9.71,M-LC,57.29,154523.0,3578.0,105879.0,-0.19,2.37,68.52,72.51,49.0,0.03,1.22,16.34,XR,NTT,IT
1,AARTIIND,487.04,919.00,37.81,M-SC,87.50,79091.0,-2245.0,74385.0,0.30,-2.76,94.05,88.69,169.0,-0.03,0.63,33.37,XR,NTT,CHEMICALS


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,59.14,80.0,-57.75,L-SC,97.92,63226.0,-50323.0,90375.0,2.68,-44.32,142.94,35.27,181.0,-0.56,0.50,54.75,XR,NTT,HOTELS
53,MASFIN,326.60,399.5,-24.12,H-SC,52.08,86220.0,-11760.0,33626.0,5.80,-12.00,39.00,22.32,102.0,-0.35,0.68,25.05,XR,ATH,FINANCE
90,VALIANTORG,512.64,838.0,-344.54,H-SC,71.88,103961.0,-29325.0,113920.0,0.65,-22.00,109.58,63.47,145.0,-0.26,0.82,74.26,XR,NTT,CHEMICALS
87,UJJIVANSFB,52.77,60.0,38.86,M-SC,42.71,120960.0,-21519.0,41042.0,0.54,-15.10,33.93,13.70,213.0,-0.52,0.96,42.77,X40N,NTT,BANKS
12,BANKINDIA,113.49,190.0,-33.09,M-MC,16.67,132065.0,2913.0,84152.0,-0.10,2.26,63.72,67.42,57.0,0.03,1.04,28.44,XR,NTT,BANKS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,718.00,-852.35,H-SC,100.00,71702.0,-23614.0,68310.0,0.92,-24.77,95.27,46.89,173.0,-0.35,0.57,41.42,X40N,NTT,MISC
94,WHIRLPOOL,1151.42,2270.00,-33.65,M-SC,98.96,166992.0,20762.0,121303.0,1.50,14.20,72.64,97.15,28.0,0.17,1.32,42.66,X40,NTT,DURABLES
21,COFFEEDAY,59.14,80.00,-57.75,L-SC,97.92,63226.0,-50323.0,90375.0,2.68,-44.32,142.94,35.27,181.0,-0.56,0.50,54.75,XR,NTT,HOTELS
56,PFIZER,4236.33,6318.30,-2.52,H-SC,96.88,151888.0,20562.0,43987.0,-1.01,15.66,28.96,49.15,94.0,0.47,1.20,25.19,X40,ATH,PHARMA
51,LTIM,5664.39,7262.39,-6.56,X-LC,95.83,150081.0,-19851.0,67792.0,-0.25,-11.68,45.17,28.21,32.0,-0.29,1.19,23.31,X200,ATH,IT


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,17.01
1,25,38.34
2,50,67.61


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    34.29
X40N    15.29
XY25    13.85
X40     13.28
AR      11.14
XR       9.85
X200     1.19
SR       1.10
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    22.64
M-SC    17.63
X-LC    15.97
H-MC     8.82
M-LC     8.27
H-LC     5.91
X-MC     5.53
M-MC     5.52
X-SC     2.81
L-SC     2.30
L-LC     1.19
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,11.55,-1.40,33.65
BANKS,10.87,-9.58,63.32
IT,9.04,-20.98,88.48
FINANCE,7.45,-17.46,70.47
MISC,7.08,-26.66,81.16
INSURANCE,6.11,0.13,29.98
PAINTS,5.47,-18.83,54.00
AUTO,4.29,-11.32,59.85
ELECTRICAL,4.09,-7.44,62.19


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3010931.0,27
X40N,1231626.0,18
AR,1181791.0,13
XR,1104630.0,14
X40,722670.0,11
XY25,463703.0,10
SR,206456.0,2
X200,67792.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2325224.0,22
M-SC,2010881.0,21
X-LC,721021.0,12
M-MC,569241.0,5
H-MC,489464.0,8
X-SC,353840.0,4
M-LC,333855.0,6
L-SC,306828.0,4
X-MC,305461.0,5


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      915808.0      6
M-SC       XY24      761891.0      6
H-SC       AR        537406.0      3
           X40N      523301.0      7
M-SC       X40N      449773.0      6
M-MC       XY24      401223.0      3
X-LC       X40       387955.0      5
M-SC       XR        303583.0      4
H-SC       XR        277304.0      4
M-LC       XY24      207831.0      4
M-SC       X40       194026.0      2
H-LC       AR        194019.0      4
H-MC       XY24      181037.0      2
L-SC       XR        155694.0      2
X-LC       AR        145304.0      2
X-MC       XY24      144874.0      2
M-SC       XY25      122832.0      1
X-SC       SR        121100.0      1
X-LC       XY25      119970.0      4
X-SC       XR        119495.0      1
           X40N      113245.0      2
M-LC       XR        105879.0      1
X-MC       X40N      103036.0      2
M-SC       AR         93420.0      1
L-SC       XY24       86638.0      1
M-SC       SR         85356.0      1
M-MC       XR         84152.0      1
           AR         83866.0      1
H-MC       XY25       75069.0      1
H-SC       X40        71405.0      2
H-MC       X40        69284.0      2
X-LC       X200       67792.0      1
L-SC       AR         64496.0      1
H-MC       AR         63280.0      1
           XR         58523.0      1
X-MC       XY25       57551.0      1
H-MC       X40N       42271.0      1
L-LC       XY25       38504.0      1
H-LC       XY25       29632.0      1
M-LC       XY25       20145.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
